In [1]:
import re

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as bs
from lxml import html


In [2]:
table= pd.read_csv('./Dirty_data_survey.csv', delimiter=',')
#table.head(5)

In [3]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27772 entries, 0 to 27771
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             27772 non-null  object 
 1   How old are you?                                                                                             

In [3]:
# first reviewing data: search how many columns had a great quantity of nulls
the_nulls= table.isnull().sum()
#the_nulls

In [4]:
#Total entries= 27,772 (100%)
the_nulls = table.isnull().mean()
the_nulls = the_nulls[the_nulls > 0].sort_values()*100 
#the_nulls


# first six columns has less than 20% of null values, so i decide to drop the rest.

In [5]:
#Droping useless columns, columns upon 20% of null values
drop_list=list(the_nulls[the_nulls >= 20].index)
table.drop(drop_list, axis=1, inplace=True)


In [ ]:
table.head(5)

In [6]:
#Beforte to manipulate data...changing column names:
table.rename(columns= {'Timestamp': 'Date',
                        'How old are you?': 'Age',
                        'What industry do you work in?': 'Workplace',
                        'Job title': 'Ocuppation',
"What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": 'Annual_salary',
                        'Please indicate the currency': 'Payment_currency',
                        'What country do you work in?': 'Work_country',
                        "If you're in the U.S., what state do you work in?": 'American_work_state',
                        'What city do you work in?': 'Workplace_city',
                        'How many years of professional work experience do you have overall?': 'Overall_experience_years',
                        'How many years of professional work experience do you have in your field?': 'Field_experience_years',
                        'What is your highest level of education completed?': 'Educational_level',
                        'What is your gender?': 'Gender',
                        'What is your race? (Choose all that apply.)': 'Race/ethnicity'}, inplace=True)
              
              
              

In [14]:
table.head(3)

,Date,Age,Workplace,Ocuppation,Annual_salary,Payment_currency,Work_country,American_work_state,Workplace_city,Overall_experience_years,Field_experience_years,Educational_level,Gender,Race/ethnicity
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,"55,000",USD,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,"54,600",GBP,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,"34,000",USD,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White


## Data manipulation for each columns 
### Date

In [7]:
table['Hour'] = pd.to_datetime(table['Date']).dt.hour
table['Date'] = pd.to_datetime(table['Date']).dt.date

### Age

In [8]:
#No changes to commit
age= table['Age'].unique().tolist()
age

['25-34', '45-54', '35-44', '18-24', '65 or over', '55-64', 'under 18']

In [9]:
table['Age'].isnull().sum()

0

### Workplace

In [10]:
table['Workplace'] = table['Workplace'].apply(lambda x: re.sub(r'(\(\w.+\))', '', str(x)))
table['Workplace'] = table['Workplace'].apply(lambda x: re.sub('-.+\w.+','',str(x)))
table['Workplace'] = table['Workplace'].apply(lambda x: re.sub('(\sor.+)','',str(x)))
table['Workplace'] = table['Workplace'].apply(lambda x: re.sub('\s/\s','/',str(x)))
table['Workplace'] = table['Workplace'].apply(lambda x: re.sub('[""]','',str(x)))
table['Workplace'] = table['Workplace'].apply(lambda x: re.sub('(:.+)','',str(x)))


In [11]:

table['Workplace'].replace('My company sells & services various types of printers, mostly kyocera.','Sales', inplace=True)
table['Workplace'].replace('I work at a property tax management company. not sure where this fits in. consulting maybe?', 'Financial', inplace=True)
table['Workplace'].replace('Language services company, unsure the broad category to use. our clients are branding agencies, and their clients are frequently  pharmaceutical companies.',
                          'Language services', inplace=True)
table['Workplace'].replace('I work in the finance function of a large global conglomerate', 'Administrative', inplace=True)
table['Workplace'].replace('Digital marketing within a book publishing company ', 'Publishing',inplace=True)
table['Workplace'].replace('I work for indeed.com','Web Services', inplace=True)

In [13]:
table['Workplace'] = table.Workplace.str.strip()
table['Workplace'] = table.Workplace.str.capitalize()
#workplaces= table['Workplace'].unique().tolist()

### Occupation 

 Because the diversity of the jobs, no changes has done.

### Annual Salary

In [14]:
for i in table['Annual_salary']:
    if len(i)<4:
        table['Annual_salary'].replace(i, i+',000', inplace=True) 

table['Annual_salary'] = table['Annual_salary'].apply(lambda x: re.sub('(,)','',str(x))) 
#then i realice that is better to take off the comma 


### Payment currency

In [15]:
table['Payment_currency'].isnull().sum()

0

In [434]:
coin= table['Payment_currency'].unique().tolist()
coin

['USD',
 'GBP',
 'CAD',
 'EUR',
 'AUD/NZD',
 'Other',
 'CHF',
 'ZAR',
 'SEK',
 'HKD',
 'JPY']

In [408]:
exc= table.loc[table.Payment_currency=='Other'][['Workplace_city', 'Work_country']]

In [411]:
exc.Workplace_city.unique().tolist()

['Pune',
 'City of buenos aires',
 'Kuala lumpur',
 'Geneva',
 'Oslo',
 'Portland',
 'Rio de janeiro',
 'Stockholm',
 'Copenhagen',
 'Budapest',
 'San fernando',
 'Bangalore',
 'Curitiba',
 'Mexico city',
 'Prague',
 'Glasgow',
 'Major',
 'London',
 '..',
 'Pietermaritzburg',
 'Seattle',
 'Batangas',
 'Undisclosed',
 'Olomouc',
 'İstanbul',
 'Denmark',
 'Beijing',
 'Hod hasharon',
 'Berkeley, ca',
 'Franklin',
 'Jerusalem',
 'Jianshui',
 'Perth',
 'Rockford',
 'Taipei',
 'Taguig city',
 'Sao paulo',
 'New delhi',
 'Singapore',
 'Changwon',
 'Bangkok',
 'Washington, dc',
 'Jakarta',
 'Christchurch',
 'Hertzliya',
 'Haikou hainan',
 'Sydney',
 'Chomutov',
 'Colombo',
 'Kraków',
 'Taguig',
 'Mackay',
 'Makati',
 'Brooklyn',
 'São paulo',
 'San nicolás de los arroyos',
 'Buenos aires',
 'Tel aviv',
 'Tijuana',
 'Metro manila',
 'Belo horizonte',
 'Warsaw',
 'Brisbane',
 'Seoul',
 'Riyadh',
 'Mumbai',
 'Tawau',
 'Georgetown',
 'Elko',
 'Kl',
 'Brno',
 'Katowice',
 'Bandung',
 'Makati city',

In [420]:
exchange= {'MXN': ['Mexico City', 'Tijuana', 'Monterrey'],
           'INR': ['Pune', 'Bangalore','New Delhi', 'Mumbai'],
           'ARS': ['City of Buenos Aires'],
           'MYR': ['Kuala Lumpur', 'Tawau', 'Georgetown', 'KL'],
           'CHF': ['Geneva'],
           'NOK': ['Oslo', 'Major'],
           'USD': ['Portland', 'Seattle','Berkeley, CA','Franklin','Rockford', 'Washington, DC', 'Elko'],
           'BRL': ['RIo de Janeiro', 'Curitiba', 'Rio de Janeiro', 'Sao Paulo'],
           'SEK': ['Stockholm'],
           'DKK': ['Copenhagen', 'Denmark'],
           'HUF': ['Budapest'],
           'TTD': ['San Fernando'],
           'CZK': ['Prague', 'olomouc', 'Brno'],
           'GBP': ['Glasgow', 'London'],
           'BDT': ['..'],
           'ZAR': ['Pietermaritzburg'],
           'PHP': ['Batangas', 'Taguig City'],
           'PLN': ['undisclosed'],
           'TRY': ['İstanbul'],
           'CYN': ['Beijing', 'Jianshui'],
           'ILS': ['Hod Hasharon', 'Jerusalem', 'Hertzliya'],
           'AUD': ['Perth'],
           'TWD': ['Taipei'],
           'SGD': ['Singapore'],
           'KRW': ['Changwon'],
           'THB': ['Bangkok'],
           'IDR': ['Jakarta'],
           'NZD': ['Christchurch'],
           'SAR': ['Riyadh']}
           

In [438]:
for value, citi in table[['Payment_currency', 'Workplace_city']].values[0:10000]:
    if value== 'Other':
        for coin,cities in exchange.items():
            for city in cities:
                if citi==city:
                    table.loc[table.Workplace_city == city, 'Payment_currency']= coin
                else:
                    pass
            
       
    

In [439]:
coin= table['Payment_currency'].unique().tolist()
coin

['USD',
 'GBP',
 'CAD',
 'EUR',
 'AUD',
 'INR',
 'Other',
 'DKK',
 'CHF',
 'AUD/NZD',
 'ZAR',
 'SEK',
 'HKD',
 'NOK',
 'HUF',
 'BRL',
 'CZK',
 'BDT',
 'PHP',
 'TRY',
 'CYN',
 'ILS',
 'JPY']

In [27]:
# Here is the code to take from a currency calculator the excange but for 27_000 values it will take 7 hours and
# for some reason it stops in the value 139. For this reason i cannot append the annual salary in dollars
usd_salary=[]
try:
    for i,j in table[['Annual_salary','Payment_currency',]].values:
        if j!= 'Other':
            URL= 'https://www.calculator.net/currency-calculator.html?eamount={total}&efrom={currency}&eto=USD&ccmajorccsettingbox=1&type=1&x=68&y=23'.format(total= i ,currency= j)
            soup = bs(requests.get(URL).content, 'lxml')
            select=soup.select('p font b')
            total= select[0].text
            usd_salary.append(total)
            print (total)
        else:
            usd_salary.append(0)

except:
    pass

#total['Annual_Salary_USD']=usd_salary

55,000
69,614.17681
34,000
62,000
60,000
62,000
33,000
50,000
112,000
45,000
47,500
62,000
100,000
52,000
25,097.11407
30,599.63816
85,000
59,000
98,000
54,000
74,000
50,000
49,409.94332
96,000
44,500
60,000
62,000
48,000
140,000
80,000
39,000
125,000
230,000
110,000
50,000
68,000
41,000
76,000
78,000
37,000
100,000
44,624.47232
65,000
187,500
110,000
66,625
144,600
200,850
94,114.17775
76,467.76942
96,000
53,000
95,000
72,000
40,782.81036
106,000
115,000
100,500
33,200
66,299.21601
52,000
61,958.50035
104,000
58,821.36109
52,000
70,000
84,000
53,000
34,000
38,000
87,000
122,000
57,374.32155
45,000
88,000
71,000
71,000
75,000
80,000
125,000
65,000
45,899.45724
55,000
90,000
105,000
47,057.08888
65,000
85,000
39,520
90,192.75368
76,000
180,000
90,000
24,607.20902
65,000
98,166
63,500
58,000
47,700
87,000
37,645.6711
39,635
53,000
47,255
196,000
34,361
150,000
75,000
20,000
107,220
61,174.21554
95,000
42,000
100,000
63,749.24617
119,000
84,149.00494
64,498.58264
75,000
145,000
74,507.057

In [28]:
len(usd_salary)

139

### Work_country

In [109]:
table['Work_country'].isnull().sum()

0

In [17]:
paises=table['Work_country'].unique().tolist()
len(paises)
#Start

366

In [18]:
table['Work_country'] = table.Work_country.str.strip()
table['Work_country'] = table.Work_country.str.capitalize()
paises=table['Work_country'].unique().tolist()
len(paises)

251

In [101]:
copy=table.copy()

#### USA

In [102]:
USA=copy[copy.Work_country.str.contains('🇺🇸|Isa|^u.s$|U.S.+|U.s|Us|U.S|usa|Usa|america|America|The U.+?|Un.+ s.+|I.s.|un.+ s.+')]



In [103]:
US= USA['Work_country'].unique().tolist()
US

['United states',
 'Us',
 'Usa',
 'U.s.',
 'United states of america',
 'U.s>',
 'Isa',
 'United state',
 'U.s.a',
 'U.s.a.',
 'America',
 'The united states',
 'United state of america',
 'United stated',
 'Usa-- virgin islands',
 'United statws',
 'U.s',
 'Unites states',
 'United sates',
 'United states of american',
 'Uniited states',
 'United sates of america',
 'United states (i work from home and my clients are all over the us/canada/pr',
 'Unted states',
 'United statesp',
 'United stattes',
 'United statea',
 'United statees',
 'Uniyed states',
 'Uniyes states',
 'United states of americas',
 'Us of a',
 'U.sa',
 'United status',
 'Canada and usa',
 'Uss',
 'Uniteed states',
 'United stares',
 'I.s.',
 'Unite states',
 'For the united states government, but posted overseas',
 'United statew',
 "I earn commission on sales. if i meet quota, i'm guaranteed another 16k min. last year i earned an additional 27k. it's not uncommon for people in my space to earn 100k+ after commissio

In [106]:
for i in copy['Work_country']:
    if i in US:
        table['Work_country'].replace(i, 'USA', inplace=True) 



In [180]:
uss=['U. s.','U.a.', 'Uxz','San francisco', 'The us', 'Is', 'Unitedstates', 'California','United y', 'I work for a uae-based organization, though i am personally in the us.']

#cannot change it

In [181]:
for i in table['Work_country']:
    if i in uss:
        table['Work_country'].replace(i, 'USA', inplace=True) 


#### United Kingdom

In [136]:
UK= copy[copy.Work_country.str.contains('u\.k|U\.k|kin|gr.+|Gr.+|uk|Uk|bri.+|Bri.+|u.k|En.+|eng.+|scot.+|Sco.+')]

In [ ]:
Uk= UK.Work_country.unique().tolist()

for i in Uk:
    if i=='Turkey' or i== 'Greece' or i== 'Ukraine' or 'Austria' in i:
        Uk.remove(i)


In [139]:
for i in Uk:
    if i== 'Ukraine':
        Uk.remove(i)

In [143]:
for i in table['Work_country']:
    if i in Uk:
        table['Work_country'].replace(i, 'United Kingdom', inplace=True) 


#### Canada

In [154]:
Can= copy[copy.Work_country.str.contains('ada|Can|can|Cs|cs')]
can= Can.Work_country.unique().tolist()

In [156]:
for i in can:
    if 'states' in i:
        can.remove(i)

In [158]:
for i in table['Work_country']:
    if i in can:
        table['Work_country'].replace(i, 'Canada', inplace=True) 


#### Netherlands

In [197]:
Net= copy[copy.Work_country.str.contains('nether|Nether|Ned.+,Nl')]
net= Net.Work_country.unique().tolist()

In [201]:
net.append('Nl')
net.append('Nederland')
net

['The netherlands', 'Netherlands', 'Nl', 'Nederland']

In [202]:
for i in table['Work_country']:
    if i in net:
        table['Work_country'].replace(i, 'Netherlands', inplace=True) 

#### New Zealand

In [203]:
Nz= copy[copy.Work_country.str.contains('new|New|nz|Nz|NZ')]
nz= Nz.Work_country.unique().tolist()

In [204]:
nz

['New zealand',
 'New zealand aotearoa',
 'Nz',
 'From new zealand but on projects across apac',
 'Aotearoa new zealand']

In [205]:
for i in table['Work_country']:
    if i in nz:
        table['Work_country'].replace(i, 'New Zealand', inplace=True) 

#### Australia

In [216]:
Au= copy[copy.Work_country.str.contains('stra')]
au= Au.Work_country.unique().tolist()

In [217]:
au

['Australia', 'Australi', 'Australian']

In [218]:
for i in table['Work_country']:
    if i in au:
        table['Work_country'].replace(i, 'Australia', inplace=True) 

#### Argentina

In [220]:
Ar= copy[copy.Work_country.str.contains('gen')]
ar= Ar.Work_country.unique().tolist()

In [221]:
ar

['Argentina',
 "I work for an us based company but i'm from argentina.",
 'Argentina but my org is in thailand']

In [222]:
for i in table['Work_country']:
    if i in ar:
        table['Work_country'].replace(i, 'Argentina', inplace=True) 

#### Czech Republic

In [228]:
Cz= table[table.Work_country.str.contains('zec')]
cz= Cz.Work_country.unique().tolist()

In [229]:
cz

['Czech republic', 'Czechia']

In [230]:
for i in table['Work_country']:
    if i in cz:
        table['Work_country'].replace(i, 'Czech Republic', inplace=True) 

#### Denmark

In [232]:
Dn= table[table.Work_country.str.contains('mark')]
dn= Dn.Work_country.unique().tolist()

In [233]:
dn

['Denmark', 'Danmark']

In [234]:
for i in table['Work_country']:
    if i in dn:
        table['Work_country'].replace(i, 'Denmark', inplace=True) 

#### China

In [236]:
Ch= table[table.Work_country.str.contains('hin')]
ch= Ch.Work_country.unique().tolist()

In [237]:
ch

['China', 'Mainland china']

In [238]:
for i in table['Work_country']:
    if i in ch:
        table['Work_country'].replace(i, 'China', inplace=True) 

#### Japan

In [239]:
Jp= table[table.Work_country.str.contains('Jap|jap')]
jp= Jp.Work_country.unique().tolist()

In [240]:
jp

['Japan', 'Japan, us gov position']

In [241]:
for i in table['Work_country']:
    if i in jp:
        table['Work_country'].replace(i, 'Japan', inplace=True) 

#### Mexico

In [244]:
Mx= table[table.Work_country.str.contains('xi')]
mx= Mx.Work_country.unique().tolist()

In [245]:
mx

['Mexico', 'México']

In [246]:
for i in table['Work_country']:
    if i in mx:
        table['Work_country'].replace(i, 'Mexico', inplace=True) 

#### Philippines

In [247]:
Ph= table[table.Work_country.str.contains('ili')]
ph= Ph.Work_country.unique().tolist()

In [248]:
ph

['Philippines', 'Remote (philippines)']

In [249]:
for i in table['Work_country']:
    if i in ph:
        table['Work_country'].replace(i, 'Philippines', inplace=True) 

#### Italy

In [253]:
It= table[table.Work_country.str.contains('taly')]
it= It.Work_country.unique().tolist()

In [254]:
it

['Italy (south)', 'Italy']

In [255]:
for i in table['Work_country']:
    if i in it:
        table['Work_country'].replace(i, 'Italy', inplace=True) 

#### Hong Kong

In [260]:
Hk= table[table.Work_country.str.contains('kon')]
hk= Hk.Work_country.unique().tolist()

In [261]:
hk

['Hong kong', 'Hong konh']

In [262]:
for i in table['Work_country']:
    if i in hk:
        table['Work_country'].replace(i, 'Hong Kong', inplace=True) 

#### Luxembourg

In [263]:
Lx= table[table.Work_country.str.contains('ux')]
lx= Lx.Work_country.unique().tolist()

In [264]:
lx

['Luxembourg', 'Luxemburg']

In [265]:
for i in table['Work_country']:
    if i in lx:
        table['Work_country'].replace(i, 'Luxembourg', inplace=True) 

#### Romania

In [266]:
Rm= table[table.Work_country.str.contains('oman')]
rm= Rm.Work_country.unique().tolist()

In [267]:
rm

['Romania', 'From romania, but for an us based company']

In [268]:
for i in table['Work_country']:
    if i in rm:
        table['Work_country'].replace(i, 'Romania', inplace=True) 

#### Pakistan

In [271]:
Pk= table[table.Work_country.str.contains('akis')]
pk= Pk.Work_country.unique().tolist()

In [272]:
pk

['Pakistan', 'Company in germany. i work from pakistan.']

In [273]:
for i in table['Work_country']:
    if i in pk:
        table['Work_country'].replace(i, 'Pakistan', inplace=True) 

#### NA

In [314]:
Na= table[table.Work_country.str.contains('Na')]
na= Na.Work_country.unique().tolist()

In [316]:
na.append('N/a (remote from wherever i want)')
na

['Na', 'N/a (remote from wherever i want)']

In [317]:
for i in table['Work_country']:
    if i in na:
        table['Work_country'].replace(i, 'Nowhere', inplace=True) 

#### Casos

In [284]:
table.loc[table.Work_country == '$2,175.84/year is deducted for benefits', 'Work_country'] = 'Canada'

In [289]:
table.loc[table.Work_country == 'Currently finance', 'Work_country'] = 'USA'

In [ ]:
table.loc[table.Work_country == 'I was brought in on this salary to help with the ehr and very quickly was promoted to current position but compensation was not altered.', 'Work_country'] = 'USA'

In [294]:
table.loc[table.Work_country == 'Worldwide (based in us but short term trips aroudn the world)', 'Work_country']= 'USA'

In [296]:
table.loc[table.Work_country == 'Y', 'Work_country']='USA'

In [299]:
table.loc[table.Work_country == 'Bonus based on meeting yearly goals set w/ my supervisor', 'Work_country'] = 'USA'

In [301]:
table.loc[table.Work_country == "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so i got a raise because a 3rd party assessment showed i was paid too little for the area we were located", 'Work_country'] = 'USA'

In [318]:
unique= table.Work_country.unique().tolist()
unique= sorted(unique)
unique

['Afghanistan',
 'Africa',
 'Argentina',
 'Australia',
 'Austria',
 'Bangladesh',
 'Belgium',
 'Bermuda',
 'Bosnia and herzegovina',
 'Brasil',
 'Brazil',
 'Bulgaria',
 'Cambodia',
 'Canada',
 'Catalonia',
 'Cayman islands',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Contracts',
 'Costa rica',
 "Cote d'ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Ecuador',
 'Eritrea',
 'Estonia',
 'Europe',
 'Finland',
 'France',
 'Germany',
 'Ghana',
 'Global',
 'Greece',
 'Hartford',
 'Hong Kong',
 'Hungary',
 'India',
 'Indonesia',
 'International',
 'Ireland',
 'Isle of man',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jersey, channel islands',
 'Jordan',
 'Kenya',
 'Kuwait',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mexico',
 'Morocco',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Northern ireland',
 'Norway',
 'Nowhere',
 'Pakistan',
 'Panamá',
 'Philippines',
 'Poland',
 'Policy',
 'Portugal',
 'Puerto rico',
 'Qatar',
 'R

In [319]:
len(unique)
#Final

111

### Workplace_city

In [321]:
copy=table.copy()

In [323]:
copy.Workplace_city.isna().sum()

75

In [327]:
copy.loc[copy.Workplace_city.isna()]['Work_country'].unique() 
#For future analysis we can occuppy the country, so, no things to do here.

array(['Spain', 'USA', 'Canada', 'Ireland', 'Finland', 'Somalia',
       'United Kingdom', 'Denmark'], dtype=object)

#### How in a loop add the name of the country in the city's column?

In [357]:
city= table.Workplace_city.unique().tolist()
len(city)
#Hardwork to categorizing all the cities, more than the countries

4773

In [360]:
table['Workplace_city'] = table.Workplace_city.str.strip()
table['Workplace_city'] = table.Workplace_city.str.capitalize()
len(city)

4773

### Overall_experience_years

In [361]:
table.Overall_experience_years.isna().sum()

0

In [370]:
table.Overall_experience_years.unique().tolist()

['5-7 years',
 '8 - 10 years',
 '2 - 4 years',
 '21 - 30 years',
 '11 - 20 years',
 '1 year or less',
 '41 years or more',
 '31 - 40 years']

In [374]:
table['Overall_experience_years'] = table['Overall_experience_years'].apply(lambda x: re.sub(r' - ', '-', str(x)))
table.Overall_experience_years.unique().tolist()

['5-7 years',
 '8-10 years',
 '2-4 years',
 '21-30 years',
 '11-20 years',
 '1 year or less',
 '41 years or more',
 '31-40 years']

### Field_experience_years

In [375]:
table.Field_experience_years.isna().sum()

0

In [376]:
table.Field_experience_years.unique().tolist()

['5-7 years',
 '2 - 4 years',
 '21 - 30 years',
 '11 - 20 years',
 '1 year or less',
 '8 - 10 years',
 '31 - 40 years',
 '41 years or more']

In [377]:
table['Field_experience_years'] = table['Field_experience_years'].apply(lambda x: re.sub(r' - ', '-', str(x)))
table.Overall_experience_years.unique().tolist()

['5-7 years',
 '8-10 years',
 '2-4 years',
 '21-30 years',
 '11-20 years',
 '1 year or less',
 '41 years or more',
 '31-40 years']

### Educational_level

In [378]:
table.Educational_level.isna().sum()

209

In [379]:
table.loc[table.Educational_level.isna()]

,Date,Age,Workplace,Ocuppation,Annual_salary,Payment_currency,Work_country,American_work_state,Workplace_city,Overall_experience_years,Field_experience_years,Educational_level,Gender,Race/ethnicity,Hour
13,2021-04-27,25-34,Law,Legal Aid Staff Attorney,52000,USD,USA,Minnesota,Saint paul,2-4 years,2-4 years,NaN,Woman,White,11
24,2021-04-27,25-34,Public library,Teen Librarian,44500,USD,USA,Florida,Bradenton,5-7 years,2-4 years,NaN,Woman,White,11
192,2021-04-27,35-44,Education,Associate Director,63000,USD,USA,Texas,El paso,11-20 years,8-10 years,NaN,Woman,"Hispanic, Latino, or Spanish origin",11
241,2021-04-27,35-44,Sales,Sales support,23050,GBP,United Kingdom,NaN,London,11-20 years,8-10 years,NaN,Woman,White,11
659,2021-04-27,45-54,Utilities & telecommunications,Financial Analyst,87360,USD,USA,Texas,Houston,21-30 years,11-20 years,NaN,Woman,Black or African American,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27174,2021-10-26,25-34,Computing,Solution Architect,36000,EUR,Latvia,NaN,Riga,11-20 years,11-20 years,NaN,Man,White,13
27289,2021-10-26,25-34,Energy,Pumper,70720,USD,USA,North Dakota,Fairfield,11-20 years,5-7 years,NaN,NaN,NaN,14
27587,2021-12-28,35-44,Education,teacher,58000,EUR,Germany,NaN,Mannheim,21-30 years,5-7 years,NaN,NaN,NaN,16
27725,2022-03-22,25-34,Engineering,R&D Engineer,96000,USD,USA,California,Redwood city,2-4 years,1 year or less,NaN,Man,NaN,18


In [380]:
table.Educational_level.unique().tolist()

["Master's degree",
 'College degree',
 'PhD',
 nan,
 'Some college',
 'High School',
 'Professional degree (MD, JD, etc.)']

In [383]:
table.loc[table.Educational_level.isna(), 'Educational_level']='Some college'

In [384]:
table.Educational_level.unique().tolist()

["Master's degree",
 'College degree',
 'PhD',
 'Some college',
 'High School',
 'Professional degree (MD, JD, etc.)']

### Gender

In [385]:
table.Gender.isna().sum()

165

In [395]:
table.loc[table.Gender.isna()]

,Date,Age,Workplace,Ocuppation,Annual_salary,Payment_currency,Work_country,American_work_state,Workplace_city,Overall_experience_years,Field_experience_years,Educational_level,Gender,Race/ethnicity,Hour
171,2021-04-27,25-34,Government and public administration,Program Analyst,117500,USD,USA,NaN,Washington dc,8-10 years,5-7 years,Master's degree,NaN,"Hispanic, Latino, or Spanish origin",11
490,2021-04-27,25-34,Hospitality & events,Senior Manager,75000,USD,USA,California,Los angeles,5-7 years,5-7 years,College degree,NaN,White,11
537,2021-04-27,25-34,Education,Online Instructor,70000,CAD,Canada,NaN,"Waterloo, on",5-7 years,5-7 years,Master's degree,NaN,White,11
647,2021-04-27,35-44,Media & digital,Assistant Editor,40000,USD,USA,New Jersey,Philly 'burbs,11-20 years,11-20 years,College degree,NaN,White,11
658,2021-04-27,25-34,Nonprofits,Prospect Research Coordinator,65000,USD,USA,New Jersey,Newark,8-10 years,8-10 years,Master's degree,NaN,White,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27289,2021-10-26,25-34,Energy,Pumper,70720,USD,USA,North Dakota,Fairfield,11-20 years,5-7 years,Some college,NaN,NaN,14
27327,2021-10-26,25-34,Business,Operationnal planning manager,33000,EUR,France,NaN,Paris,2-4 years,2-4 years,Master's degree,NaN,White,15
27339,2021-10-26,35-44,Engineering,Sr. Infrastructure Engineer,110000,USD,USA,North Carolina,Raleigh,8-10 years,8-10 years,College degree,NaN,White,16
27587,2021-12-28,35-44,Education,teacher,58000,EUR,Germany,NaN,Mannheim,21-30 years,5-7 years,Some college,NaN,NaN,16


In [387]:
table.Gender.unique().tolist()

['Woman',
 'Non-binary',
 'Man',
 nan,
 'Other or prefer not to answer',
 'Prefer not to answer']

In [399]:
table.loc[table.Gender.isna(), 'Gender']='Prefer not to answer'
table.loc[table.Gender == 'Other or prefer not to answer', 'Gender']='Prefer not to answer'


In [401]:
table.Gender.unique().tolist()

['Woman', 'Non-binary', 'Man', 'Prefer not to answer']

### Race/Ethnicity

In [402]:
table['Race/ethnicity'].isna().sum()

164

In [403]:
table.loc[table['Race/ethnicity'].isna()]

,Date,Age,Workplace,Ocuppation,Annual_salary,Payment_currency,Work_country,American_work_state,Workplace_city,Overall_experience_years,Field_experience_years,Educational_level,Gender,Race/ethnicity,Hour
445,2021-04-27,25-34,Property,HR Manager,82400,USD,USA,Washington,Seattle,8-10 years,5-7 years,College degree,Woman,NaN,11
479,2021-04-27,25-34,Education,Finance assistant,37024,USD,USA,Michigan,Kalamazoo,5-7 years,2-4 years,PhD,Prefer not to answer,NaN,11
530,2021-04-27,25-34,Computing,Data Engineer,95000,USD,USA,Maryland,Baltimore,8-10 years,2-4 years,College degree,Man,NaN,11
773,2021-04-27,35-44,Education,Curriculum & Teaching Specialist,40460,USD,USA,Arkansas,Fayetteville,11-20 years,8-10 years,Master's degree,Prefer not to answer,NaN,11
1159,2021-04-27,35-44,Government and public administration,Chief of Staff,102000,CAD,Canada,NaN,Ottawa,11-20 years,1 year or less,Some college,Woman,NaN,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27599,2022-01-05,25-34,Health care,Director,120000,USD,USA,Georgia,Atlanta,5-7 years,5-7 years,PhD,Woman,NaN,18
27684,2022-02-25,35-44,"Accounting, banking & finance",Controller,150000,USD,USA,Texas,Austin,11-20 years,11-20 years,Master's degree,Woman,NaN,19
27725,2022-03-22,25-34,Engineering,R&D Engineer,96000,USD,USA,California,Redwood city,2-4 years,1 year or less,Some college,Man,NaN,18
27730,2022-03-27,18-24,Nan,chief executive official,50000,ZAR,Spain,NaN,Castellon,8-10 years,11-20 years,Some college,Prefer not to answer,NaN,10


In [404]:
table['Race/ethnicity'].unique().tolist()

['White',
 'Hispanic, Latino, or Spanish origin, White',
 'Asian or Asian American, White',
 'Asian or Asian American',
 'Another option not listed here or prefer not to answer',
 'Hispanic, Latino, or Spanish origin',
 'Middle Eastern or Northern African',
 'Hispanic, Latino, or Spanish origin, Middle Eastern or Northern African, White',
 'Black or African American',
 'Black or African American, White',
 nan,
 'Black or African American, Hispanic, Latino, or Spanish origin, White',
 'Native American or Alaska Native',
 'Native American or Alaska Native, White',
 'Hispanic, Latino, or Spanish origin, Another option not listed here or prefer not to answer',
 'Black or African American, Middle Eastern or Northern African, Native American or Alaska Native, White',
 'White, Another option not listed here or prefer not to answer',
 'Black or African American, Native American or Alaska Native, White',
 'Asian or Asian American, Another option not listed here or prefer not to answer',
 'Middl

In [405]:
len(table['Race/ethnicity'].unique().tolist())

49

Due to problems to identify each human been , I prefeer to skip the cleaning.

In [441]:
Data_clean=table

In [442]:
Data_clean.to_csv('Data_clean')